# 模型加载

In [12]:
import torch
from spirl.modules.subnetworks import VQCDTPredictor
import spirl.configs.local

# 模型配置
codebook = 16
class HyperParameters:
    def __init__(self):
        self.codebook_K = codebook
        self.feature_learning_depth = -1
        self.decision_depth = 6
        self.num_intermediate_variables = 20
        self.greatest_path_probability = False
        self.beta_fl = False
        self.beta_dc = False
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hp = HyperParameters()

tree = VQCDTPredictor(hp, 60, codebook)
tree.load_model('/home/zuo/project/xrl/spirl/experiments/cdt_model/16_-1+60+6+0_s6_2copy.pth')

use SDT


# 参数打印

In [13]:
num_params = 0
for key, v in tree.state_dict().items():
    print(key, v.reshape(-1).shape[0])
    num_params+=v.reshape(-1).shape[0]
print('Total number of parameters in model: ', num_params)

dc_leaves 1024
dc_inner_nodes.weight 3843
Total number of parameters in model:  4867


# 决策树绘制

In [14]:
import numpy as np

def get_binary_index(tree, max_depth):
    """
    Get binary index for tree nodes:
    From

    0
    1 2
    3 4 5 6 

    to 

    '0'
    '00' '01' 
    '000' '001' '010' '011'

    """
    index_list = []
    for layer_idx in range(0, max_depth+1):
        index_list.append([bin(i)[2:].zfill(layer_idx+1) for i in range(0, np.power(2, layer_idx))])
    return np.concatenate(index_list)

from matplotlib.patches import ConnectionPatch

def _add_arrow(ax_parent, ax_child, xyA, xyB, color='black', linestyle=None):
    '''Private utility function for drawing arrows between two axes.'''
    con = ConnectionPatch(xyA=xyA, xyB=xyB, coordsA='data', coordsB='data',
                            axesA=ax_child, axesB=ax_parent, arrowstyle='<|-',
                            color=color, linewidth=tree.decision_depth, linestyle=linestyle)
    ax_child.add_artist(con)

In [15]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.patches import ConnectionPatch
import matplotlib.patches as patches
from matplotlib import rcParams

def draw_tree(tree, savepath, type):
    if type == 'DM':
        inner_node_num = tree.num_dc_inner_nodes # 内节点数
        leaf_node_num = tree.num_dc_leaves # 叶节点数
        max_depth = tree.decision_depth # 树深度
        inner_nodes_name='dc_inner_nodes.weight'
        leaf_nodes_name='dc_leaves'
        input_shape = (tree.num_intermediate_variables,)
    
    inner_nodes = tree.state_dict()[inner_nodes_name] # (31, 21): 20特征权重+1偏置
    leaf_nodes = tree.state_dict()[leaf_nodes_name] # (32, 16): 16动作概率
    binary_indices = get_binary_index(tree, max_depth)
    inner_indices = binary_indices[:inner_node_num]
    leaf_indices = binary_indices[inner_node_num:]
    output_dim = tree.output_dim

    kernels = dict([(node_idx, node_value.cpu().numpy().reshape(input_shape)) for node_idx, node_value in zip (inner_indices, inner_nodes[:, 1:])]) # 内节点权重
    biases = dict([(node_idx, node_value.cpu().numpy().squeeze()) for node_idx, node_value in zip (inner_indices, inner_nodes[:, :1])]) # 内节点偏置
    leaves = dict([(leaf_idx, np.array([leaf_dist.cpu().numpy()])) for leaf_idx, leaf_dist in zip (leaf_indices, leaf_nodes)]) # 叶节点概率
    n_leaves = leaf_node_num

    # 图片尺寸设置
    fig = plt.figure(figsize=(4*n_leaves, n_leaves), facecolor='white')
    gs = GridSpec(max_depth+1, n_leaves*2, height_ratios=[1]*max_depth+[0.5]) # 建立网格
    rcParams['font.size'] = 32 # 字体大小

    gcx = [list(np.arange(1, 2**(i+1), 2) * (2**(max_depth+1) // 2**(i+1)))for i in range(max_depth+1)]
    gcx = list(itertools.chain.from_iterable(gcx)) # 每个节点的列坐标（包括内节点和叶节点，从顶层开始
    
    axes = {} # TODO
    
    # draw inner nodes
    for pos, key in enumerate(sorted(kernels.keys(), key=lambda x:(len(x), x))): # 执行31次，输出0 0、1 00、2 01...
        ax = plt.subplot(gs[len(key)-1, gcx[pos]-2:gcx[pos]+2]) # 画格子
        axes[key] = ax
        kernel_image = kernels[key]
        max_index = np.argmax(np.abs(kernel_image)) # 最大的权重
        max_kernel = kernel_image[max_index]

        # ax.text((kernel_image.shape[0])//4, (kernel_image.shape[0])//4, 's[{}] > {:.3f}'.format(max_index, -biases[key]/max_kernel))
        # vector_image = np.ones(((kernel_image.shape[0])//2, kernel_image.shape[0]))
        # color = 'Blues'
        # ax.imshow(vector_image, vmin=0, vmax=1, cmap=plt.get_cmap(color))
        # 定义矩形的参数：(x, y, width, height)

        rect = patches.Rectangle((0, 0), 1, 1, facecolor='#2D8634')
        ax.add_patch(rect)
        ax.text(0.5, 0.5, 's[{}] > {:.3f}'.format(max_index, -biases[key]/max_kernel), ha='center', va='center', fontsize=42, color='white', weight='bold')

        ax.axis('off')
        if type!='FL':  # 设置划分到的动作
            digits = set([np.argmax(leaves[k]) for k in leaves.keys()
                        if k.startswith(key)])
            title = ','.join(str(digit) for digit in digits)
            plt.title('{}'.format(title))
        
    # draw leaf nodes
    for pos, key in enumerate(sorted(leaves.keys(), key=lambda x:(len(x), x))):
        ax = plt.subplot(gs[len(key)-1, gcx[len(kernels)+pos]-1:gcx[len(kernels)+pos]+1])
        axes[key] = ax
        leaves_image = leaves[key].squeeze(0)
        leaves_image = np.exp(leaves_image) / np.sum(np.exp(leaves_image), axis=0)
        max_index = np.argmax(leaves_image)
        max_leaf = leaves_image[max_index]

        rect = patches.Rectangle((0, 0), 1, 1, facecolor='#1DB12A') # #00DB12
        ax.add_patch(rect)
        ax.text(0.5, 0.5, 'a[{}] = {:.2f}'.format(max_index, max_leaf), ha='center', va='center', fontsize=32, color='white', weight='bold')

        ax.axis('off')
        if type!='FL':  # feature learning tree do not have titile indicating the classification 
            plt.title('{}'.format(np.argmax(leaves[key])), y=-.5)

    # add arrows indicating flow
    for pos, key in enumerate(sorted(axes.keys(), key=lambda x:(len(x), x))):
        children_keys = [k for k in axes.keys() if len(k) == len(key) + 1 and k.startswith(key)]
        for child_key in children_keys:
            # p_rows, p_cols = axes[key].get_images()[0].get_array().shape
            # c_rows, c_cols = axes[child_key].get_images()[0].get_array().shape

            # # distinguish with green and red color
            # color = 'green' if (key in path and child_key in path) else 'red'
            # _add_arrow(axes[key], axes[child_key],
            #            (c_cols//2, 1), (p_cols//2, p_rows-1), color)

            # distinguish with solid or dotted lines
            linestyle = None
            _add_arrow(axes[key], axes[child_key], (0.5, 1), (0.5, 0), color='black', linestyle=linestyle)
draw_tree(tree, '/home/zuo/project/xrl/spirl/experiments/cdt_model/16+-1+5+20.png', 'DM')